In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [14]:
...
# load ascii text and covert to lowercase
filename = "rrr.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [15]:
print(raw_text)

the project gutenberg ebook of alice's adventures in wonderland
    
this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever. you may copy it, give it away or re-use it under the terms
of the project gutenberg license included with this ebook or online
at www.gutenberg.org. if you are not located in the united states,
you will have to check the laws of the country where you are located
before using this ebook.

title: alice's adventures in wonderland

author: lewis carroll

release date: june 27, 2008 [ebook #11]
                most recently updated: february 4, 2024

language: english

credits: arthur dibianca and david widger


*** start of the project gutenberg ebook alice's adventures in wonderland ***
[illustration]




alice’s adventures in wonderland

by lewis carroll

the millennium fulcrum edition 3.0

contents

 chapter i.     down the rabbit-hole
 chapter ii.    the pool of tea

In [16]:
...
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
print(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(char_to_int)

['\n', ' ', '!', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ù', '—', '‘', '’', '“', '”', '•', '™']
{'\n': 0, ' ': 1, '!': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55, 'ù': 56, '—': 57, '‘': 58, '’': 59, '“': 60, '”': 61, '•': 62, '™': 63}


In [17]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  163916
Total Vocab:  64


In [20]:
...
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
print(dataX[0])
print(dataY[0])

Total Patterns:  163816
[49, 37, 34, 1, 45, 47, 44, 39, 34, 32, 49, 1, 36, 50, 49, 34, 43, 31, 34, 47, 36, 1, 34, 31, 44, 44, 40, 1, 44, 35, 1, 30, 41, 38, 32, 34, 6, 48, 1, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 1, 38, 43, 1, 52, 44, 43, 33, 34, 47, 41, 30, 43, 33, 0, 1, 1, 1, 1, 0, 49, 37, 38, 48, 1, 34, 31, 44, 44, 40, 1, 38, 48, 1, 35, 44, 47, 1, 49, 37, 34, 1, 50, 48, 34, 1, 44, 35, 1, 30, 43]
54


In [7]:
...
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

print(X)
print(y)

[[[0.765625]
  [0.578125]
  [0.53125 ]
  ...
  [0.015625]
  [0.46875 ]
  [0.671875]]

 [[0.578125]
  [0.53125 ]
  [0.015625]
  ...
  [0.46875 ]
  [0.671875]
  [0.84375 ]]

 [[0.53125 ]
  [0.015625]
  [0.703125]
  ...
  [0.671875]
  [0.84375 ]
  [0.6875  ]]

 ...

 [[0.6875  ]
  [0.015625]
  [0.578125]
  ...
  [0.6875  ]
  [0.625   ]
  [0.75    ]]

 [[0.015625]
  [0.578125]
  [0.53125 ]
  ...
  [0.625   ]
  [0.75    ]
  [0.1875  ]]

 [[0.578125]
  [0.53125 ]
  [0.640625]
  ...
  [0.75    ]
  [0.1875  ]
  [0.      ]]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [8]:
X.shape

(163816, 100, 1)

In [9]:
y.shape

(163816, 64)

In [10]:
...
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

C:\Users\RAMU\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
...
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
  13/1280 ━━━━━━━━━━━━━━━━━━━━ 9:56 471ms/step - loss: 3.9273

KeyboardInterrupt: 